## **🔰 Упрощенный код для Whisper [ver.2 Google Colab] 08-09-2023**

In [1]:
!ls -ahl

total 16K
drwxr-xr-x 1 root root 4.0K Sep 11 13:22 .
drwxr-xr-x 1 root root 4.0K Sep 12 17:09 ..
drwxr-xr-x 4 root root 4.0K Sep 11 13:21 .config
drwxr-xr-x 1 root root 4.0K Sep 11 13:22 sample_data


In [2]:
#@markdown Проврека выделенной видео-карты (нужна T4 или выше)
!nvidia-smi
!nvidia-smi --query-gpu=name --format=csv,noheader,nounits


Tue Sep 12 17:09:51 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   59C    P8    10W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [3]:
# Установка yt-dlp
!pip install -q yt-dlp

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 17.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.4/194.4 kB 19.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 32.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.9/129.9 kB 17.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 48.7 MB/s eta 0:00:00


In [4]:
# Функция очистки ссылок
import re

def clean_youtube_url(url: str) -> str:
    """
    Преобразует любую ссылку на видео YouTube в формат короткой ссылки (https://youtu.be/ID_ВИДЕО).

    Параметры:
        url (str): Исходная ссылка на видео на YouTube.

    Возвращает:
        str: Короткая ссылка на видео или None, если исходная ссылка не соответствует формату YouTube.

    Пример:
        >>> clean_youtube_url("https://www.youtube.com/watch?v=dQw4w9WgXcQ")
        "https://youtu.be/dQw4w9WgXcQ"
    """

    # Регулярное выражение для поиска идентификаторов видео YouTube:
    # 1. (?:https?:\/\/)? - необязательный протокол (http или https).
    # 2. (?:www\.)? - необязательный префикс "www".
    # 3. (?:youtube\.com\/(?:watch\?v=|embed\/)|youtu\.be\/) - паттерн для длинных (стандартных и embed) и коротких ссылок YouTube.
    # 4. ([a-zA-Z0-9_-]{11}) - идентификатор видео, состоящий из 11 символов.
    pattern = r"(?:https?:\/\/)?(?:www\.)?(?:youtube\.com\/(?:watch\?v=|embed\/)|youtu\.be\/)([a-zA-Z0-9_-]{11})"

    # Поиск совпадения с помощью регулярного выражения
    match = re.search(pattern, url)
    if match:
        # Если найдено совпадение, извлекаем идентификатор видео
        video_id = match.group(1)
        return f"https://youtu.be/{video_id}"
    else:
        return None

In [28]:
# Список ссылок для загрузки

urls_list = [

"https://youtu.be/YYSED9790dE?si=lx6rHxDT-Fz5O2rW",
"https://youtu.be/W9kGG0-xpx4?si=bYDsbGmeZ8q7DXjx",
"https://youtu.be/RW7aZQqN2as?si=grqmAH38yaqAvShh",
"https://youtu.be/BNMuN8Hvdx4?si=LibOi-HbL3sI-b6A",
"https://youtu.be/jTX-C5ymcwM?si=19qZsCY7bzvjzfsz",
"https://youtu.be/_VbRAQRFkO0?si=Ptbg5zs0IpEX2Scw"

]

In [29]:
# Создаем список "очищенных" коротких ссылок на видео YouTube.
# Все недопустимые или неподходящие ссылки будут проигнорированы.
cleaned_urls = set(filter(None, map(clean_youtube_url, urls_list)))

# Выводим результат
print(cleaned_urls)

{'https://youtu.be/BNMuN8Hvdx4', 'https://youtu.be/W9kGG0-xpx4', 'https://youtu.be/jTX-C5ymcwM', 'https://youtu.be/YYSED9790dE', 'https://youtu.be/RW7aZQqN2as', 'https://youtu.be/_VbRAQRFkO0'}


In [7]:
# Функция загрузки видео в формате m4a (аудиофайл) с YouTube в директоррию /content/videos/
import subprocess

def download_video(url: str) -> None:
    """
    Загружает видео с YouTube в формате m4a (аудиофайл) и сохраняет в директории /content/audios/.

    Параметры:
        url (str): Ссылка на видео на YouTube.

    Пример:
        >>> download_video("https://www.youtube.com/watch?v=dQw4w9WgXcQ")
        ...
    """

    # Команда для yt-dlp, которая:
    # 1. Использует опцию "-x" для извлечения аудио.
    # 2. Устанавливает формат аудио в "m4a".
    # 3. Определяет путь для сохранения файла.
    cmd = [
        "yt-dlp",
        "-x",
        "--audio-format", "m4a",
        "-o", "/content/audios/%(title)s.%(ext)s",
        url
    ]

    try:
        # Инициализация подпроцесса с заданной командой.
        # stdout=subprocess.PIPE позволяет читать вывод в реальном времени.
        process = subprocess.Popen(cmd, stdout=subprocess.PIPE, stderr=subprocess.STDOUT, text=True)

        # Чтение вывода команды в реальном времени и его вывод на экран.
        for line in process.stdout:
            print(line.strip())

        # Ожидание завершения подпроцесса и получение кода завершения.
        return_code = process.wait()

        # Если процесс завершился с ошибкой (не нулевой код завершения), генерируем исключение.
        if return_code != 0:
            raise subprocess.CalledProcessError(return_code, cmd)

    # Обработка исключений при выполнении команды.
    except subprocess.CalledProcessError as e:
        print(f"Ошибка при обработке ссылки {url}:")
        print(str(e))

In [30]:
# Перебор каждой очищенной ссылки из списка cleaned_urls.
# Для каждой ссылки будет вызвана функция download_video,
# которая загрузит видео в формате m4a и сохранит его в директории /content/audios/.
for url in cleaned_urls:
    download_video(url)

[youtube] Extracting URL: https://youtu.be/BNMuN8Hvdx4
[youtube] BNMuN8Hvdx4: Downloading webpage
[youtube] BNMuN8Hvdx4: Downloading ios player API JSON
[youtube] BNMuN8Hvdx4: Downloading android player API JSON
[youtube] BNMuN8Hvdx4: Downloading m3u8 information
[info] BNMuN8Hvdx4: Downloading 1 format(s): 251
[download] Destination: /content/audios/Новый Kia Sorento ｜ Замена щеток стеклоочистителей.webm

[download]   0.1% of    1.57MiB at  263.31KiB/s ETA 00:06
[download]   0.2% of    1.57MiB at  679.17KiB/s ETA 00:02
[download]   0.4% of    1.57MiB at    1.40MiB/s ETA 00:01
[download]   0.9% of    1.57MiB at    2.69MiB/s ETA 00:00
[download]   1.9% of    1.57MiB at    1.23MiB/s ETA 00:01
[download]   3.9% of    1.57MiB at    1.71MiB/s ETA 00:00
[download]   7.9% of    1.57MiB at    2.15MiB/s ETA 00:00
[download]  15.8% of    1.57MiB at    2.98MiB/s ETA 00:00
[download]  31.8% of    1.57MiB at    4.59MiB/s ETA 00:00
[download]  63.6% of    1.57MiB at    7.44MiB/s ETA 00:00
[download]

In [9]:
# Установка whisper
!pip install -q git+https://github.com/openai/whisper.git

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 13.7 MB/s eta 0:00:00


In [10]:
import os
from typing import List

def _construct_whisper_command(input_path: str, output_dir: str) -> List[str]:
    """
    Формирование команды для программы whisper.

    Args:
    - input_path (str): Путь к исходному аудиофайлу.
    - output_dir (str): Путь к директории, где сохранить результаты транскрибации.

    Returns:
    - List[str]: Список аргументов для команды whisper.

    Команда whisper используется для автоматической транскрибации аудиозаписей.
    В данной функции мы формируем список аргументов для этой команды:
    1. `--model large-v2`: использование улучшенной большой модели (версии 2) для транскрибации.
    2. `--language ru`: указание языка речи на русском.
    3. `--device cuda`: использование графического процессора (GPU) для ускорения транскрибации.
    4. `--output_format txt`: формат вывода результатов транскрибации в текстовом файле.
    """
    return [
        'whisper',
        input_path,
        '--model', "large-v2",
        '--language', 'ru',
        '--device', 'cuda',
        '--output_format', 'all',
        '--output_dir', output_dir
    ]


def transcribe_audio_files(input_directory: str, output_directory: str) -> None:
    """
    Транскрибирование всех аудиофайлов из указанной директории с помощью whisper.

    Args:
    - input_directory (str): Директория с исходными аудиофайлами.
    - output_directory (str): Директория для сохранения результатов транскрибации.

    Для каждого файла из `input_directory` запускается процесс транскрибации.
    Результаты сохраняются в поддиректории `output_directory`, где каждая поддиректория соответствует одному аудиофайлу.
    """

    # Проверка наличия выходной директории и её создание при отсутствии
    if not os.path.exists(output_directory):
        os.makedirs(output_directory)

    # Формирование списка аудиофайлов с расширением .m4a
    files = [f for f in os.listdir(input_directory) if os.path.isfile(os.path.join(input_directory, f)) and f.endswith('.m4a')]

    # Для каждого аудиофайла:
    for file in files:
        input_path = os.path.join(input_directory, file)

        # Имя поддиректории формируется на основе имени файла без расширения
        subdir_name = os.path.splitext(file)[0]
        subdir_path = os.path.join(output_directory, subdir_name)

        # Информирование пользователя о текущем файле
        print(f"Транскрибирование файла: {file}...")

        # Формирование команды для whisper
        cmd = _construct_whisper_command(input_path, subdir_path)

        # Запуск процесса транскрибации и вывод результатов в реальном времени
        with subprocess.Popen(cmd, stdout=subprocess.PIPE, text=True) as process:
            for line in iter(process.stdout.readline, ''):
                print(line, end='')  # Вывод строки в реальном времени
            print(f"\nТранскрибирование файла {file} завершено.")


In [31]:
transcribe_audio_files('/content/audios', '/content/out')

Транскрибирование файла: Новый Kia Sorento ｜ Замена щеток стеклоочистителей.m4a...
[00:00.000 --> 00:21.600]  Для того, чтобы дворники лобового стекла работали эффективно, Вам нужно следить за состоянием
[00:21.600 --> 00:25.960]  щеток стеклоочистителя и периодически протирать их. Если щетки стеклоочистителя
[00:26.300 --> 00:28.560]  начинают оставлять полосы или больше не очищают стекло
[00:28.560 --> 00:33.540]  должным образом, их нужно будет заменить. При замене стеклоочистителей выключите двигатель.
[00:33.540 --> 00:38.120]  Поднимите переключатель режимов стеклоочистителя в верхнее положение и удерживайте
[00:38.120 --> 00:40.920]  его в течение двух секунд для перехода в сервисный режим.
[00:40.920 --> 00:46.160]  Со стеклоочистителями в верхнем положении поднимите привод стеклоочистителя,
[00:46.160 --> 00:50.820]  откройте кронштейн щетки стеклоочистителя и потяните ее вниз, чтобы отсоединить.
[00:50.820 --> 01:02.700]  установите новую щетку в обратном порядке. При замене 

In [32]:
!ls -ahl /content/audios

total 17M
drwxr-xr-x 3 root root 4.0K Sep 12 18:08  .
drwxr-xr-x 1 root root 4.0K Sep 12 18:06  ..
drwxr-xr-x 2 root root 4.0K Sep 12 18:07  .ipynb_checkpoints
-rw-r--r-- 1 root root 1.6M Oct 28  2020 'Новый Kia Sorento ｜ Автоматическая система полного привода.m4a'
-rw-r--r-- 1 root root 2.3M Oct 28  2020 'Новый Kia Sorento ｜ Возможности трансформации салона.m4a'
-rw-r--r-- 1 root root 1.8M Oct 28  2020 'Новый Kia Sorento ｜ Датчики и радары.m4a'
-rw-r--r-- 1 root root 6.8M Oct 27  2020 'Новый Kia Sorento ｜ Дизайн и технологии.m4a'
-rw-r--r-- 1 root root 2.1M Oct 28  2020 'Новый Kia Sorento ｜ Замена щеток стеклоочистителей.m4a'
-rw-r--r-- 1 root root 2.0M Oct 28  2020 'Новый Kia Sorento ｜ Система Terrain Mode.m4a'


In [34]:
!ls -ahl /content/out

total 96K
drwxr-xr-x 24 root root 4.0K Sep 12 18:22  .
drwxr-xr-x  1 root root 4.0K Sep 12 18:06  ..
drwxr-xr-x  2 root root 4.0K Sep 12 18:22  .ipynb_checkpoints
drwxr-xr-x  2 root root 4.0K Sep 12 17:30 'Kia K5 ｜ Ассистент движения в полосе'
drwxr-xr-x  2 root root 4.0K Sep 12 17:34 'Kia K5 ｜ Безопасная парковка'
drwxr-xr-x  2 root root 4.0K Sep 12 17:45 'Kia K5 ｜ Замена батарейки в Smart Key'
drwxr-xr-x  2 root root 4.0K Sep 12 17:40 'Kia K5 ｜ Замена салонного фильтра'
drwxr-xr-x  2 root root 4.0K Sep 12 17:43 'Kia K5 ｜ Интеллектуальный круиз-контроль'
drwxr-xr-x  2 root root 4.0K Sep 12 17:29 'Kia K5 ｜ Камеры для контроля слепых зон с отображением на панель приборов'
drwxr-xr-x  2 root root 4.0K Sep 12 17:46 'Kia K5 ｜ Камеры кругового обзора'
drwxr-xr-x  2 root root 4.0K Sep 12 17:21 'Kia K5 ｜ Монитор кругового обзора'
drwxr-xr-x  2 root root 4.0K Sep 12 17:31 'Kia K5 ｜ Система безопасного выхода'
drwxr-xr-x  2 root root 4.0K Sep 12 17:19 'Kia K5 ｜ Система контроля внимания водител

In [35]:
!zip -r /content/out.zip /content/out

updating: content/out/ (stored 0%)
updating: content/out/Kia K5 ｜ Камеры кругового обзора/ (stored 0%)
updating: content/out/Kia K5 ｜ Камеры кругового обзора/Kia K5 ｜ Камеры кругового обзора.vtt (deflated 58%)
updating: content/out/Kia K5 ｜ Камеры кругового обзора/Kia K5 ｜ Камеры кругового обзора.json (deflated 85%)
updating: content/out/Kia K5 ｜ Камеры кругового обзора/Kia K5 ｜ Камеры кругового обзора.txt (deflated 58%)
updating: content/out/Kia K5 ｜ Камеры кругового обзора/Kia K5 ｜ Камеры кругового обзора.tsv (deflated 55%)
updating: content/out/Kia K5 ｜ Камеры кругового обзора/Kia K5 ｜ Камеры кругового обзора.srt (deflated 60%)
updating: content/out/Kia K5 ｜ Ассистент движения в полосе/ (stored 0%)
updating: content/out/Kia K5 ｜ Ассистент движения в полосе/Kia K5 ｜ Ассистент движения в полосе.vtt (deflated 60%)
updating: content/out/Kia K5 ｜ Ассистент движения в полосе/Kia K5 ｜ Ассистент движения в полосе.srt (deflated 62%)
updating: content/out/Kia K5 ｜ Ассистент движения в полосе/

In [36]:
from google.colab import files
files.download("/content/out.zip")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [37]:

#import os

directory = '/content/out'
for filename in os.listdir(directory):
    f = os.path.join(directory, filename)
    if os.path.isfile(f):
        print(f)

In [48]:
#создаем папку для вывода текстовый файлов с заголовками для БД
os.mkdir("/content/out_txt2")

In [39]:
#Пройдемся по всем файлам каталога out и найдем текстовые
#Преобразуем файл следующим образом
#1. Из названия файла сформируем его заголовок (заменив символ '|' на точку)
#2. Добавим заловок в начало, оформив в "#"
#3. Из содержимого файла уберу символы переноса строк (все равно оцифровывать)
directory = '/content/out'
directory_final = '/content/out_txt'

for root, dirs, files in os.walk(directory):
    for file in files:
        if file.endswith('.txt'):
             TextHeader = file.replace( " ｜ ", '. ')

             with open (os.path.join(root, file )  , 'r') as f:
                old_data = f.read()
             #print(old_data)
             #new_data = f"#{TextHeader}#\n {old_data}"
             #print(new_data)
             print(f"{directory_final}/{file}")
             with open (f"{directory_final}/{file}" , 'w') as f:
                f.write(f"#{file.replace( ' ｜ ', '. ')}#\n{old_data}" )




/content/out_txt/Kia K5 ｜ Камеры кругового обзора.txt
/content/out_txt/Новый Kia Sorento ｜ Автоматическая система полного привода.txt
/content/out_txt/Kia K5 ｜ Ассистент движения в полосе.txt
/content/out_txt/Kia K5 ｜ Система предотвращения столкновения при выезде с парковки задним ходом.txt
/content/out_txt/Kia K5 ｜ Система безопасного выхода.txt
/content/out_txt/Kia K5 ｜ Интеллектуальный круиз-контроль.txt
/content/out_txt/Новый Kia Sorento ｜ Возможности трансформации салона.txt
/content/out_txt/Новый Kia Sorento ｜ Дизайн и технологии.txt
/content/out_txt/Kia K5 ｜ Безопасная парковка.txt
/content/out_txt/Краш-тесты Kia K5 ｜ K5 safety test.txt
/content/out_txt/Kia K5 ｜ Замена батарейки в Smart Key.txt
/content/out_txt/Новый Kia Sorento ｜ Замена щеток стеклоочистителей.txt
/content/out_txt/Kia K5 ｜ Системы защиты детей.txt
/content/out_txt/Kia K5 ｜ Система предотвращения выезда из полосы движения.txt
/content/out_txt/Новый Kia Sorento ｜ Датчики и радары.txt
/content/out_txt/Kia K5 ｜ Ка

In [40]:
!zip -r /content/out_txt.zip /content/out_txt

updating: content/out_txt/ (stored 0%)
updating: content/out_txt/Kia K5 ｜ Камеры для контроля слепых зон с отображением на панель приборов.txt (deflated 60%)
updating: content/out_txt/Kia K5 ｜ Системы защиты детей.txt (deflated 63%)
updating: content/out_txt/Kia K5 ｜ Система предотвращения фронтальных столкновений.txt (deflated 66%)
updating: content/out_txt/Kia K5 ｜ Система предотвращения столкновения при выезде с парковки задним ходом.txt (deflated 65%)
updating: content/out_txt/Kia K5 ｜ Камеры кругового обзора.txt (deflated 58%)
updating: content/out_txt/Kia K5 ｜ Интеллектуальный круиз-контроль.txt (deflated 65%)
updating: content/out_txt/Kia K5 ｜ Система предотвращения выезда из полосы движения.txt (deflated 64%)
updating: content/out_txt/Kia K5 ｜ Замена салонного фильтра.txt (deflated 59%)
updating: content/out_txt/Краш-тесты Kia K5 ｜ K5 safety test.txt (deflated 66%)
updating: content/out_txt/Kia K5 ｜ Система безопасного выхода.txt (deflated 58%)
updating: content/out_txt/Kia K5 

In [47]:
from google.colab import files
files.download("/content/out_txt.zip")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [46]:
#Собираем все файлы из директории в один
import fileinput, glob, os

# каталог текстовых файлов
path = '/content/out_txt'
# паттерн поиска файлов по расширению
pattern = '*.txt'

glob_path = os.path.join(path, pattern)
list_files = glob.glob(glob_path)
# расширение нового файла установим как '.all'
new_file = '/content/video_database.txt'

if list_files:
    # открываем список файлов 'list_files' на чтение
    # и новый общий файл 'new_file' на запись
    with fileinput.FileInput(files=list_files) as fr, open(new_file, 'w') as fw:
        # читаем данные построчно
        for line in fr:
            # определяем первую строку нового файла
            if fr.isfirstline():
                # название читаемого файла
                file_name = fr.filename()
                # дописываем строку с названием файла
                #fw.write(f'\n {file_name}\n')
                fw.write(f'\n')
            # если нужно, то здесь обрабатываем каждую строку 'line'
            # после обработки дописываем в общий файл
            #fw.write(line)
            fw.write(f"{line.replace( '.txt', '. ')}")
